# Configuración Inicial

## Habilitar GPU (solo para Google Colab)

En la barra superior:

1.   Entorno de ejecución
2.   Cambiar tipo de entorno de ejecución -> GPU


## Subir *dataset* (solo para Google Colab)

In [ ]:
### QUITAR COMENTARIOS Y EJECUTAR PARA USAR EN GOOGLE COLAB ###
#from google.colab import files

#uploaded = files.upload()  # Seleccionar el archivo desde el sistema local

# 1 Procesamiento de datos

## 1.1 Carga y exploración inicial

### 1.1.1 Carga del dataset y visualización inicial

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset
df = pd.read_csv("Housing.csv")  # Asegurarse de tener el archivo en el directorio local, o subirlo manualmente a Google Colab

In [ ]:
# Visualizar primeras filas
print("Primeras 5 filas:")
print(df.head())

# Verificar valores nulos
print("\nValores faltantes por columna:")
df.isnull().sum()

No hay valores faltantes en ninguna columna. Por tanto, no es necesario hacer tratamiento de valores nulos.

### 1.1.2 Análisis de distribuciones y outliers

Graficar histogramas y diagramas de caja:

In [ ]:
# Configurar estilo de gráficos
sns.set_theme(style="whitegrid")

# Histogramas para variables numéricas
numerical_cols = ['area', 'price', 'bedrooms', 'bathrooms', 'stories', 'parking']
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribución de {col}')
plt.tight_layout()
plt.show()

# Boxplots para detectar outliers
plt.figure(figsize=(15, 8))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(y=df[col])
    plt.title(f'Boxplot de {col}')
plt.tight_layout()
plt.show()

- **Histogramas**:

    Muestran la distribución de cada variable numérica.

    Si la distribución tiene una cola larga a la derecha (**sesgo positivo**), significa que hay valores extremadamente altos. En este caso, sucede con `area` y `price`.

- **Boxplots**:

    Identifican valores **atípicos** (_outliers_) en cada variable.

    Los puntos fuera de los "bigotes" (líneas horizontales) son _outliers_. De nuevo, `area` y `price` tienen varios, lo que sugiere que hay propiedades con áreas mucho más grandes que el resto, y con un precio también mucho mayor a los demás.

## 1.2 Preprocesamiento de datos

### 1.2.1 Codificación de variables categóricas

- **Binarias**: `1` para "_yes_", `0` para "_no_".

- ¿Por qué es mejor One-Hot Encoding que Label Encoding en este caso?

    1. Evita jerarquías artificiales:

        La red neuronal podría interpretar erróneamente que unfurnished (0) < semi-furnished (1) < furnished (2), lo que no necesariamente refleja la realidad en los precios.

    2. Flexibilidad del modelo:

        Con One-Hot, cada categoría se trata como una característica independiente, permitiendo al modelo aprender contribuciones no lineales.

    3. Ejemplo práctico:

        Una propiedad semi-furnished no es el "punto medio" entre unfurnished y furnished en términos de precio. One-Hot captura mejor esta relación.

In [ ]:
# Variables binarias (yes/no)
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
df[binary_cols] = df[binary_cols].replace({'yes': 1, 'no': 0})

# One-Hot Encoding para furnishingstatus
df = pd.get_dummies(df, columns=['furnishingstatus'], prefix='furnishing', dtype=int)

In [ ]:
# Verificar cambios
df.head(10)

### 1.2.2 Normalización de Variables Numéricas

`StandardScaler` centra las variables en 0 con **desviación estándar** 1. De esta forma, evita que variables como `area` (valores grandes) dominen el modelo.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Separar variables numéricas
numerical_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
X_numerical = df[numerical_cols]

# Escalado (usando StandardScaler)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numerical)

# Reemplazar columnas originales con valores escalados
df[numerical_cols] = X_scaled

In [ ]:
# Verificar cambios
df[numerical_cols].head()

## 1.3 Análisis de correlación

### 1.3.1 Matriz de correlación

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calcular matriz de correlación
corr_matrix = df.corr()

# Matriz de correlación para todas las variables
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Matriz de Correlación")
plt.show()

También se puede evaluar sólo la columna de la **matriz de correlación** en función de `price`:

In [ ]:
# Filtrar solo las correlaciones con 'price'
price_corr = corr_matrix[['price']].sort_values(by='price', ascending=False)

# Gráfico de correlaciones con 'price'
plt.figure(figsize=(8, 10))
sns.heatmap(price_corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlación de Variables con el Precio (price)")
plt.show()

Variables Fuertemente Correlacionadas con `price`:

- Alta correlación **positiva** (`> 0.4`): `area`, `bathrooms`, `airconditioning`: A mayor área, número de baños y/o con aire acondicionado, mayor precio.

- Correlación **negativa** (`< -0.3`): `furnishing_unfurnished`: Las propiedades no amuebladas tienden a ser más baratas.

- **Poca** o casi ninguna correlación:

    - `hotwaterheating` (`0.093`): Saber si una casa tiene o no agua caliente, aporta muy poca información sobre el precio de la misma.

    - `furnishing_semi-furnished` (`0.064`): Que una casa esté "semiamueblada" tiene un impacto mínimo en el precio.

### 1.3.2 Eliminación de variables

Con la información que se ha obtenido al realizar el análisis anterior, se pueden sacar las siguientes **conclusiones**:

- `hotwaterheating`: Se puede **eliminar** esta columna, ya que su correlación con `price` es muy baja, y no aporta información significativa al modelo.

- `furnishing_semi-furnished`: **No** se va a eliminar esta columna, ya que podría carecer de lógica que una vivienda tuviera valor 0 en las otras dos categorías de `furnishing`.

In [ ]:
# Eliminar la columna hotwaterheating
df.drop('hotwaterheating', axis=1, inplace=True)

# Verificar el dataset actualizado
df.columns

## 1.4 División Train-Test

Creación de datos sintéticos (Data Augmentation):

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Separar características (X) y variable objetivo (y)
X = df.drop('price', axis=1)  # Todas las columnas excepto 'price'
y = df['price']  # Columna 'price' como variable objetivo

def augment_data(X, y, num_samples=350):
    np.random.seed(42)
    
    # Interpolación entre puntos reales
    idx1 = np.random.randint(0, len(X), num_samples)
    idx2 = np.random.randint(0, len(X), num_samples)
    
    alpha = np.random.uniform(0, 1, num_samples).reshape(-1, 1)
    X_aug = alpha * X.iloc[idx1].values + (1 - alpha) * X.iloc[idx2].values
    y_aug = alpha.flatten() * y.iloc[idx1].values + (1 - alpha.flatten()) * y.iloc[idx2].values
    
    # Agregar ruido gaussiano
    noise_X = np.random.normal(0, 0.01, X_aug.shape)
    noise_y = np.random.normal(0, 0.01, y_aug.shape)
    
    X_aug += noise_X
    y_aug += noise_y
    
    X_aug = pd.DataFrame(X_aug, columns=X.columns)
    y_aug = pd.Series(y_aug, name=y.name)
    
    return X_aug, y_aug

# Aplicar aumentación
X_aug, y_aug = augment_data(X, y, num_samples=500)
X = pd.concat([X, X_aug], axis=0)
y = pd.concat([y, y_aug], axis=0)

Se utiliza `train_test_split` para dividir el conjunto de datos en 80% datos de entrenamiento y 20% datos de prueba.

In [ ]:
# División: 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Verificar tamaños de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

In [ ]:
X_train.head(), y_train.head()

# 2 Model Planning

Asegurarse de que `TensorFlow` esté usando la GPU:

In [ ]:
import tensorflow as tf

print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))

## 2.1 Construcción del modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import HeNormal

# Función para crear el modelo con opciones de regularización y Dropout
def create_model(optimizer, dropout_rate=0.0, l2_reg=0.0, n_units=64):
    model = Sequential([
        Dense(n_units, activation='relu', input_shape=(X_train.shape[1],),
              kernel_initializer=HeNormal(), kernel_regularizer=l2(l2_reg)),  # He normal initialization
        BatchNormalization(),
        Dropout(dropout_rate),  # Dropout para evitar sobreajuste
        Dense(n_units, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(dropout_rate),
        Dense(n_units // 2, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(l2_reg)),
        Dense(1, activation='linear')  # Capa de salida
    ])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

## 2.2 Comparativa de optimizadores y validación cruzada

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Configuración de K-Fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Diccionarios para almacenar métricas
metrics_adam = {'mae': [], 'rmse': [], 'r2': []}
metrics_sgd  = {'mae': [], 'rmse': [], 'r2': []}

# Iterar sobre cada partición del K-Fold
for train_index, val_index in kf.split(X_train):
    X_train_fold = X_train.iloc[train_index]
    y_train_fold = y_train.iloc[train_index]
    X_val_fold = X_train.iloc[val_index]
    y_val_fold = y_train.iloc[val_index]

    # --- Modelo con Adam y Gradient Clipping ---
    model_adam = create_model(Adam(learning_rate=0.001, clipvalue=0.5))
    model_adam.fit(
        X_train_fold, 
        y_train_fold, 
        validation_data=(X_val_fold, y_val_fold),
        epochs=100, 
        batch_size=32, 
        verbose=0
    )
    y_pred_adam = model_adam.predict(X_val_fold).flatten()

    # Cálculo de métricas para Adam
    metrics_adam['mae'].append(mean_absolute_error(y_val_fold, y_pred_adam))
    metrics_adam['rmse'].append(np.sqrt(mean_squared_error(y_val_fold, y_pred_adam)))
    metrics_adam['r2'].append(r2_score(y_val_fold, y_pred_adam))

    # --- Modelo con SGD y Gradient Clipping ---
    model_sgd = create_model(SGD(learning_rate=0.01, momentum=0.9, clipvalue=0.5))
    model_sgd.fit(
        X_train_fold,
        y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        epochs=100,
        batch_size=32,
        verbose=0
    )
    y_pred_sgd = model_sgd.predict(X_val_fold).flatten()

    # Cálculo de métricas para SGD
    metrics_sgd['mae'].append(mean_absolute_error(y_val_fold, y_pred_sgd))
    metrics_sgd['rmse'].append(np.sqrt(mean_squared_error(y_val_fold, y_pred_sgd)))
    metrics_sgd['r2'].append(r2_score(y_val_fold, y_pred_sgd))

# Resultados promedio
print("Resultados Adam (Cross-Validation):")
print(f"MAE: {np.mean(metrics_adam['mae']):.4f} ± {np.std(metrics_adam['mae']):.4f}")
print(f"RMSE: {np.mean(metrics_adam['rmse']):.4f} ± {np.std(metrics_adam['rmse']):.4f}")
print(f"R²: {np.mean(metrics_adam['r2']):.4f} ± {np.std(metrics_adam['r2']):.4f}")

print("\nResultados SGD (Cross-Validation):")
print(f"MAE: {np.mean(metrics_sgd['mae']):.4f} ± {np.std(metrics_sgd['mae']):.4f}")
print(f"RMSE: {np.mean(metrics_sgd['rmse']):.4f} ± {np.std(metrics_sgd['rmse']):.4f}")
print(f"R²: {np.mean(metrics_sgd['r2']):.4f} ± {np.std(metrics_sgd['r2']):.4f}")

# Evaluación final en test set (opcional)
final_model = create_model(Adam(learning_rate=0.0001, clipvalue=1.0))
final_model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=0)
y_test_pred = final_model.predict(X_test).flatten()

print("\nEvaluación final en Test Set:")
print(f"MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")
print(f"R²: {r2_score(y_test, y_test_pred):.4f}")

## 2.3 Entrenamiento del modelo

In [ ]:
# Dividir datos en entrenamiento y validación (80%-20%)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Crear el modelo con el optimizador Adam
model_adam = create_model(optimizer=Adam(learning_rate=0.001), dropout_rate=0.3, l2_reg=0.01)

# Entrenar el modelo
history = model_adam.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=350,
    batch_size=32,
    verbose=1
)

## 2.4 Definición de la arquitectura final del modelo

In [ ]:
# Arquitectura del modelo con optimizador como argumento
def create_model(optimizer, dropout_rate, l2_reg):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),                                                               # Capa de entrada (12 features)
        Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(l2_reg)),     # Primera capa oculta
        BatchNormalization(),                                                                           # Normalización por lotes
        Dropout(dropout_rate),                                                                          # Regularización por Dropout
        Dense(64, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(l2_reg)),     # Segunda capa oculta
        BatchNormalization(),                                                                           # Ayuda con la estabilidad
        Dropout(dropout_rate),                                                                          # Regularización por Dropout
        Dense(32, activation='relu', kernel_initializer=HeNormal(), kernel_regularizer=l2(l2_reg)),     # Tercera capa oculta
        Dense(1, activation='linear')                                                                   # Capa de salida con activación lineal
    ])
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# 3 Model building and selection

## 3.1 Experimentar con distintos hiperparámetros

In [ ]:
# Experimentación con diferentes hiperparámetros
def experiment_with_hyperparameters():
    results = {}
    for lr in [0.01, 0.001, 0.0001]:
        for batch_size in [16, 32, 64]:
            for dropout in [0.2, 0.3]:
                for l2_reg in [0.1, 0.01]:
                    print(f"Entrenando con lr={lr}, batch_size={batch_size}, dropout={dropout}, l2_reg={l2_reg}")
                    model = create_model(optimizer=Adam(learning_rate=lr), dropout_rate=dropout, l2_reg=l2_reg)
                    history = model.fit(
                        X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=350,
                        batch_size=batch_size,
                        verbose=0
                    )
                    val_loss = history.history['val_loss'][-1]
                    results[(lr, batch_size, dropout, l2_reg)] = val_loss
    return results

# Ejecutar experimentos
hyperparameter_results = experiment_with_hyperparameters()

Para obtener la mejor combinación de hiperparámetros:

In [ ]:
# Ordenar resultados del experimento por menor pérdida de validación (val_loss)
sorted_results = sorted(hyperparameter_results.items(), key=lambda x: x[1])

# Mostrar las 5 mejores combinaciones
print("Top 5 mejores combinaciones de hiperparámetros:")
for params, val_loss in sorted_results[:5]:
    print(f"LR: {params[0]}, Batch Size: {params[1]}, Dropout: {params[2]}, L2: {params[3]} --> Val Loss: {val_loss:.4f}")

In [ ]:
# Obtener la mejor combinación de hiperparámetros
best_hyperparams = sorted_results[0][0]  # La combinación con menor val_loss

# Crear y entrenar el modelo con la mejor combinación
best_model = create_model(optimizer=Adam(learning_rate=best_hyperparams[0]), 
                          dropout_rate=best_hyperparams[2], 
                          l2_reg=best_hyperparams[3])

history = best_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=400,
    batch_size=best_hyperparams[1],
    verbose=1
)

# Mostrar métricas finales en entrenamiento y validación
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"\nMejor combinación de hiperparámetros:")
print(f"LR: {best_hyperparams[0]}, Batch Size: {best_hyperparams[1]}, Dropout: {best_hyperparams[2]}, L2: {best_hyperparams[3]}")
print(f"Train Loss: {final_train_loss:.4f}, Val Loss: {final_val_loss:.4f}")

## 3.2 Calcular porcentaje de error

In [ ]:
# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"RMSE: {rmse:.4f}")

# RMSE como porcentaje del rango de los valores reales
rmse_range_percentage = (rmse / (np.max(y_test) - np.min(y_test))) * 100

print(f"RMSE (% sobre rango): {rmse_range_percentage:.2f}%")

# 4 Presentación de resultados

## 4.1 Evaluación de predicciones

### 4.1.1 Gráfico de dispersión entre valores reales y predicciones

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Predicciones del modelo
y_pred = best_model.predict(X_test)

# Gráfico de dispersión
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.5, label="Predicciones vs Reales")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r', label="Línea Ideal (y=x)")
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Gráfico de dispersión: Predicciones vs Valores Reales")
plt.legend()
plt.show()

- Los puntos están generalmente alineados y cercanos a la línea roja (línea ideal), lo que indica que esas predicciones son precisas.

- Las predicciones que se encuentran muy dispersas respecto a la línea, pueden indicar un sesgo.

### 4.1.2 Gráfico de residuales

In [ ]:
residuals = y_test - y_pred.flatten()
plt.figure(figsize=(8,6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicciones")
plt.ylabel("Residuales")
plt.title("Gráfico de residuales")
plt.show()

- Los puntos están dispersos aleatoriamente, lo que indica que el modelo está bien ajustado.

- Si hubiera una tendencia clara (por ejemplo, un patrón en forma de U), el modelo podría necesitar ajustes.

### 4.1.3 Boxplot de errores

In [ ]:
plt.figure(figsize=(6,4))
plt.boxplot(residuals, vert=False, patch_artist=True)
plt.xlabel("Error de Predicción")
plt.title("Boxplot de Errores")
plt.show()

- Los puntos que se encuentran muy alejados, podrían ser _outliers_ en los errores.

### 4.1.4 Curva de aprendizaje

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='Pérdida en Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en Validación')
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.title("Curva de Aprendizaje")
plt.legend()
plt.show()

- Si la pérdida de validación es mucho mayor que la de entrenamiento → Sobreajuste.

- Si ambas pérdidas son altas → El modelo podría ser demasiado simple.

## 4.2 Análisis de errores

### 4.2.1 Predicciones con errores mayores al 10%

In [ ]:
# Asegurar que y_test y y_pred sean 1D
y_test = np.ravel(y_test)  # Convierte a 1D si es necesario
y_pred = np.ravel(y_pred)  # Convierte a 1D si es necesario

# Calcular el error relativo
errores_relativos = np.abs((y_test - y_pred) / y_test)

# Filtrar los casos con errores > 10%
errores_significativos = errores_relativos > 0.10

# Mostrar ejemplos de predicciones con alto error
errores_df = pd.DataFrame({
    "Real": y_test,
    "Predicción": y_pred,
    "Error Relativo (%)": errores_relativos * 100
})

errores_df_significativos = errores_df[errores_significativos].sort_values(by="Error Relativo (%)", ascending=False)

print(errores_df_significativos.head(10))  # Ver los 10 peores casos

### 4.2.2 Distribución de los errores relativos

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(errores_relativos * 100, bins=30, edgecolor="black", alpha=0.7)
plt.axvline(x=10, color="red", linestyle="--", label="Error del 10%")
plt.xlabel("Error Relativo (%)")
plt.ylabel("Frecuencia")
plt.title("Distribución del Error Relativo en %")
plt.legend()
plt.show()

# 5 Serialización del modelo

In [ ]:
# Guardado en formato Keras (.keras)
final_model.save("housing_price_model_v1.keras")

In [ ]:
# Guardado en formato HDF5 (.h5)
final_model.save('housing_price_model_v1.h5')

In [ ]:
import tensorflow as tf

# Cargar el modelo keras y revisar su shape
loaded_model = tf.keras.models.load_model('housing_price_model_v1.keras')
print(loaded_model.summary())